
### 파이썬 3.5나 그 이전 버전에서는 keys, values, items, popitem등 

딕셔너리가 제공하는 모든 메서드가 이터레이션 순서에 의존해서 임의로 실행순서가 정해지는 것처럼 보임

### 3.5이전의 딕셔너리 구현
: 내장 hash 함수와 파이썬 인터프리터가 시작할 때 초기화되는 난수 씨앗값(seed)을 사용하는 해시 테이블 알고리즘으로 만들어짐
  
  인터프리터 실행 시마다 달라지는 난수 seed값과 hash가 어우러지며 **딕셔너리의 순서가 실행시마다 달라짐**


In [ ]:
baby_names = {
    'cat': 'kitten',
    'dog': 'puppy',
}

In [ ]:
print(baby_names)

In [ ]:
print(list(baby_names.keys()))
print(list(baby_names.values()))
print(list(baby_names.items()))
print(list(baby_names.popitem()))

### 파이썬 3.5이전에서
함수에 대한 ```**kwargs``` 인자 역시 함수 호출시 사용한 인자순서가 뒤죽박죽인 것처럼 보임 

In [ ]:
int(list(baby_names.popitem()))
def my_func(**kwargs):
    for key, value in kwargs.items():
        print('%s = %s' % (key, value))

In [ ]:
my_func(goose='gosling', kangaroo='joey')

### 클래스도 인스턴스 딕셔너리에 dict 타입 사용 3.5이전은 object필드 난수 같은 동작 



In [88]:
class MyClass:
    def __init__(self):
        self.alligator = 'hatchling'
        self.elephant = 'calf'     

In [ ]:
a = MyClass()
for key,value in a.__dict__.items():
    print('%s = %s' % (key, value))

### [OrderedDict()](https://sangwoo0727.github.io/python/Python-11_OrderedDict/)

Python 3.7이전에는 삽입 순서대로의 출력을 보장받기 위해 사용했었다.

3.7이후 Dict가 순서를 보장하게 되었지만 [최소최근사용(least-recently-used)](https://gomguard.tistory.com/115) 캐시구현 등

키 삽입과 popitem호출을 매우 자주 처리해야할 때 dict보다 권장됨


### Python에서는 프로그래머가 list, dict등의 표준 프로토콜 흉내내는 커스텀 컨테이너 타입 쉽게 정의 가능
: 객체의 동작이 객체의 실질적인 타입을 결정하는 덕 타이핑에 의존하는 파이썬의 특성


> Python에서 Class는 일종의 컨테이너로 속성과 기능을 함께 캡슐화한다. 
  내장 컨테이너 타입으로는 리스트, 튜플, 딕셔너리, 셋을 제공하며. 이를 상속하여 커스텀 컨테이너를 정의할 수 있다.


### 귀여운 동물 콘테스트

In [1]:
votes ={
    'otter': 1281,
    'polar bear': 587,
    'fox': 863
}

In [17]:
def populate_ranks(votes, ranks):
    names = list(votes.keys())
    names.sort(key=votes.get, reverse=True)
    for i, name in enumerate(names, 1):
        ranks[name] = i

In [24]:
def get_winner(ranks):
    return next(iter(ranks))

In [4]:
ranks = {}
populate_ranks(votes, ranks)
print(ranks)
winner = get_winner(ranks)
print(winner)

{'otter': 1, 'fox': 2, 'polar bear': 3}
otter


### collections.abc 모듈 활용한 새로운 클래스 정의 

In [5]:
from collections.abc import MutableMapping

In [27]:
class SortedDict(MutableMapping):
    def __init__(self):
        self.data = {}
    
    def __getitem__(self, key):
        return self.data[key]
    
    def __setitem__(self, key, value):
        self.data[key] = value
        
    def __delitem__(self, key):
        del self.data[key]
        
    def __iter__(self):
        keys = list(self.data.keys())
        keys.sort()
        
        for key in keys:
            yield key
            
    def __len__(self):
        return len(self.data)

In [25]:
ranks = {}
sorted_ranks = SortedDict()
populate_ranks(votes, sorted_ranks)
print(sorted_ranks.data)
winner = get_winner(sorted_ranks)
print(winner)

{'otter': 1, 'fox': 2, 'polar bear': 3}
fox


###  문제점
#### : get_winner의 구현이 populate_ranks의 삽입순서에 맞게 딕셔너리를 이터레이션 한다고 가정한 것

dict가 아닌 SortedDict를 사용했으므로 틀린 가정


### 해결법 1. ranks 딕셔너리가 어떤 특정 순서로 이터레이션 된다는 가정 X

가장 보수적이고 튼튼한 해법


In [30]:
def get_winner(ranks):
    for name, rank in ranks.items():
        if rank == 1:
            return name
        
winner = get_winner(sorted_ranks)
print(winner)

otter



### 해결법 2. 함수 맨 앞에 ranks 타입이 우리가 원하는 타입인지 검사하는 코드 추가

실행성능이 더 좋음 


In [31]:
def get_winner(ranks):
    if not isinstance(ranks, dict):
        raise TypeError('dict 인스턴스가 필요합니다.')
    return next(iter(ranks))

get_winner(sorted_ranks)

TypeError: dict 인스턴스가 필요합니다.

### 해결법 3. 타입 어노테이션 사용해 get_winner에 전달되는 값을 dict인스턴스로 강제 

정적 타입 안정성과 런타임 성능을 가장 잘 조합해줌


In [32]:
from typing import Dict, MutableMapping

In [52]:
def populate_ranks(votes: Dict[str, int], 
                   ranks: Dict[str, int]) -> None:
    names = list(votes.keys())
    names.sort(key=votes.get, reverse=True)
    for i, name in enumerate(names, 1):
        ranks[name] = i

In [66]:
def get_winner1(ranks: Dict[str, int]) -> str:
    return next(iter(ranks))

In [67]:
class SortedDict(MutableMapping[str, int]):
    def __init__(self):
        self.data = {}
    
    def __getitem__(self, key):
        return self.data[key]
    
    def __setitem__(self, key, value):
        self.data[key] = value
        
    def __delitem__(self, key):
        del self.data[key]
        
    def __iter__(self):
        keys = list(self.data.keys())
        keys.sort()
        
        for key in keys:
            yield key
            
    def __len__(self):
        return len(self.data)

In [55]:
votes ={
    'otter': 1281,
    'polar bear': 587,
    'fox': 863
}

In [87]:
sorted_ranks = SortedDict()
populate_ranks(votes, sorted_ranks)
print(sorted_ranks.data)
winner = get_winner1(sorted_ranks)
print(winner)

{'otter': 1, 'fox': 2, 'polar bear': 3}
fox


In [69]:
x =['빨강', '주황', '노랑']

In [78]:
odd = x[::2]

In [75]:
odd2 = odd[::1]

In [80]:
print(odd)
print(id(odd))

['빨강', '노랑']
1899452419264


In [79]:
print(odd2)
print(id(odd2))

['빨강', '노랑']
1899452419456


In [83]:
def generate_csv():
    yield('날짜', '제조사', '모델', '연식', '가격')

In [85]:
for i in (generate_csv()

<generator object generate_csv at 0x000001BA402D9510>
